In [ ]:
import requests
import pdb
import re
import os
import random
import torch
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0"}
base_url = 'https://fr.trustpilot.com/review/www.cdiscount.com'

In [ ]:
def fetch_url(url, headers, max_attempts=10):
    for attempt in range(max_attempts):
        timeout = random.randint(1, 10)
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raise en exception pour les HTTP erreurs
            print(f'Successfully fetched {url}')
            return response
        except requests.RequestException as e:
            print(f'Attempt {attempt + 1} failed for {url} with timeout {timeout}: {e}')
    return None

response = fetch_url(base_url, headers)
contenu = response.text

def extract_comments_and_total(contenu):
    comments = re.findall(r'"reviewBody":"(.*?)"', contenu)

    rating = re.findall(r'"ratingValue":"(.*?)"', contenu)
    matches = re.findall(r'"reviewBody":"(.*?)"\s*,\s*"reviewRating":\{[^}]*?"ratingValue":"(.*?)"\}',contenu)
    ratings = [rating for comment, rating in matches if comment in comments]

    nb_views_match = re.search(r"(?:[\d\s]+) sur ([\d\s]+) avis", contenu)
    total_reviews = int(nb_views_match.group(1).replace('\u202f', '').replace(' ', '')) if nb_views_match else len(comments)
    return comments, ratings, total_reviews


comments, ratings, total_reviews = extract_comments_and_total(contenu)
print(ratings)

Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com
['1', '1', '1', '1', '1', '4', '1', '4', '5', '5', '1', '5', '1', '1', '1', '4', '4', '1']


In [ ]:
comments, ratings, total_reviews = extract_comments_and_total(contenu)

all_data = [{"comment": c, "rating": float(r)} for c, r in zip(comments, ratings)]

# Calcul du nombre total de pages
total_pages = math.ceil(total_reviews / 20) if total_reviews else 1
print(f"Nombre total d'avis : {total_reviews} => {total_pages} pages estimées.")

# Récupération des pages suivantes
for page in range(2, 50 + 1):
    url = f"{base_url}?page={page}"
    response = fetch_url(url, headers)  # 👈 tu dois refetch le contenu de la page !
    contenu = response.text

    page_comments = re.findall(r'"reviewBody":"(.*?)"', contenu)
    page_ratings = re.findall(r'"reviewRating":{"@type":"Rating","bestRating":"5","worstRating":"1","ratingValue":"(.*?)"', contenu)

    if not page_comments:
        print(f"Aucun commentaire trouvé à la page {page}, arrêt.")
        break

    all_data.extend(
        {"comment": c, "rating": float(r)}
        for c, r in zip(page_comments, page_ratings)
    )
    print(f"Page {page} : {len(page_comments)} commentaires trouvés.")

print(f"\nTotal de commentaires récupérés : {len(all_data)}")

all_data



Nombre total d'avis : 268427 => 13422 pages estimées.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=2
Page 2 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=3
Page 3 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=4
Page 4 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=5
Page 5 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=6
Page 6 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=7
Page 7 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=8
Page 8 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=9
Page 9 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot

[{'comment': 'Je suis profondément mécontent de Cdiscount suite à l’achat et l’installation de ma machine à laver. J’ai payé un supplément pour une installation professionnelle, mais celle-ci a été catastrophique. La vis qui maintient le tambour n’a pas été retirée, une erreur élémentaire. Dès la première utilisation, les vibrations provoquées ont endommagé non seulement mon sol et mon mur, mais aussi les pieds mêmes de la machine.Face à ces dommages évidents causés par leur négligence, j’ai demandé un dédommagement. Leur \\',
  'rating': 1.0},
 {'comment': "je suis en situation handicape j ai commandé un matelas qui n'est jamais arrivée malgres un confirmation de livraison et j espere qu'ils me rembourseront , je dors sur le sol  depuis 5 jours a fuire c discount je déconseille vivement donc dorenavant je ferais mes achat sur amazon",
  'rating': 1.0},
 {'comment': 'J’ai commandé un Samsung  il y a un mois et demi, qui est resté bloqué sur la plate-forme, Cdiscount m’a remboursée, et 

# 2. Nettoyage structurel
[texte du lien](https://)

In [ ]:

# 1. Création DataFrame
df = pd.DataFrame(all_data)

# 2. Nettoyage structurel
df.dropna(subset=["comment", "rating"], inplace=True)
df.drop_duplicates(inplace=True)
df["comment"] = df["comment"].str.strip()
df = df[df["comment"].str.len() > 0]

# 3. Création du label binaire (sentiment)
df["label"] = df["rating"].apply(lambda x: 1 if x >= 4 else 0)

# 4. Vérification des classes
print(df["label"].value_counts(normalize=True))  # % positif / négatif


label
0    0.60423
1    0.39577
Name: proportion, dtype: float64


 ## 2.1 **🧼–** NETTOYAGE DU TEXTE

In [ ]:
from bs4 import BeautifulSoup
import string
import re

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)  # facultatif
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # emojis, caractères spéciaux
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df["clean_comment"] = df["comment"].apply(clean_text)


# 3 🧠 PRÉTRAITEMENT NLP (ex : spaCy lemmatization)

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 76.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

nlp = spacy.load("fr_core_news_sm")

def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

df["lemmatized"] = df["clean_comment"].apply(lemmatize)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["lemmatized"])
y = df["label"]

In [ ]:
from sklearn.utils import class_weight
import numpy as np

weights = class_weight.compute_class_weight("balanced", classes=np.unique(y), y=y)
print(dict(zip(np.unique(y), weights)))

{np.int64(0): np.float64(0.8275), np.int64(1): np.float64(1.2633587786259541)}


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

# Tester les métriques

In [ ]:
from transformers import pipeline

# Choix d’un sous-échantillon (sinon très long)
sample_df = df.sample(100, random_state=42)

models = {
    "tblard/tf-allocine": lambda r: "positif" if r["label"] == "POSITIVE" else "negatif",
    "nlptown/bert-base-multilingual-uncased-sentiment": lambda r: "positif" if int(r["label"].split()[0]) >= 4 else "negatif",
    "cardiffnlp/twitter-xlm-roberta-base-sentiment": lambda r: "positif" if r["label"] == "Positive" else "negatif"
}

results = {}

# Vraies étiquettes : 1 → "positif", 0 → "negatif"
true_labels = sample_df["label"].apply(lambda x: "positif" if x == 1 else "negatif")


for model_name, label_fn in models.items():
    print(f"▶️ Testing model: {model_name}")
    try:
        if model_name == "tblard/tf-allocine":
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
            clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, framework="tf")
        else:
            clf = pipeline("sentiment-analysis", model=model_name)

        preds = []
        for text in sample_df["comment"]:
            try:
                pred = clf(text)[0]
                preds.append(label_fn(pred))
            except:
                preds.append("erreur")

        sample_df[f"pred_{model_name}"] = preds

        print(f"\n📊 Résultats : {model_name}")
        print(classification_report(true_labels, preds, zero_division=0))

    except Exception as e:
        print(f"⛔ Erreur pour le modèle {model_name} : {e}")


▶️ Testing model: tblard/tf-allocine


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
Device set to use 0



📊 Résultats : tblard/tf-allocine
              precision    recall  f1-score   support

     negatif       0.97      0.95      0.96        61
     positif       0.93      0.95      0.94        39

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

▶️ Testing model: nlptown/bert-base-multilingual-uncased-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : nlptown/bert-base-multilingual-uncased-sentiment
              precision    recall  f1-score   support

     negatif       0.95      0.95      0.95        61
     positif       0.92      0.92      0.92        39

    accuracy                           0.94       100
   macro avg       0.94      0.94      0.94       100
weighted avg       0.94      0.94      0.94       100

▶️ Testing model: cardiffnlp/twitter-xlm-roberta-base-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : cardiffnlp/twitter-xlm-roberta-base-sentiment
              precision    recall  f1-score   support

     negatif       0.61      1.00      0.76        61
     positif       0.00      0.00      0.00        39

    accuracy                           0.61       100
   macro avg       0.30      0.50      0.38       100
weighted avg       0.37      0.61      0.46       100



In [ ]:
sample_df[["comment","rating","label","clean_comment","lemmatized"]].to_csv("nlp_data_clean.csv")


In [ ]:
for model_name, label_fn in models.items():
    print(f"▶️ Testing model: {model_name}")
    try:
        if model_name == "tblard/tf-allocine":
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
            clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, framework="tf")
        else:
            clf = pipeline("sentiment-analysis", model=model_name)

        preds = []
        for text in sample_df["clean_comment"]:
            try:
                pred = clf(text)[0]
                preds.append(label_fn(pred))
            except:
                preds.append("erreur")

        sample_df[f"pred_{model_name}"] = preds

        print(f"\n📊 Résultats : {model_name}")
        print(classification_report(true_labels, preds, zero_division=0))

    except Exception as e:
        print(f"⛔ Erreur pour le modèle {model_name} : {e}")

▶️ Testing model: tblard/tf-allocine


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
Device set to use 0



📊 Résultats : tblard/tf-allocine
              precision    recall  f1-score   support

     negatif       0.97      0.95      0.96        61
     positif       0.93      0.95      0.94        39

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100

▶️ Testing model: nlptown/bert-base-multilingual-uncased-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : nlptown/bert-base-multilingual-uncased-sentiment
              precision    recall  f1-score   support

     negatif       0.90      0.92      0.91        61
     positif       0.87      0.85      0.86        39

    accuracy                           0.89       100
   macro avg       0.89      0.88      0.88       100
weighted avg       0.89      0.89      0.89       100

▶️ Testing model: cardiffnlp/twitter-xlm-roberta-base-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : cardiffnlp/twitter-xlm-roberta-base-sentiment
              precision    recall  f1-score   support

     negatif       0.61      1.00      0.76        61
     positif       0.00      0.00      0.00        39

    accuracy                           0.61       100
   macro avg       0.30      0.50      0.38       100
weighted avg       0.37      0.61      0.46       100



In [ ]:

for model_name, label_fn in models.items():
    print(f"▶️ Testing model: {model_name}")
    try:
        if model_name == "tblard/tf-allocine":
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
            clf = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, framework="tf")
        else:
            clf = pipeline("sentiment-analysis", model=model_name)

        preds = []
        for text in sample_df["lemmatized"]:
            try:
                pred = clf(text)[0]
                preds.append(label_fn(pred))
            except:
                preds.append("erreur")

        sample_df[f"pred_{model_name}"] = preds

        print(f"\n📊 Résultats : {model_name}")
        print(classification_report(true_labels, preds, zero_division=0))

    except Exception as e:
        print(f"⛔ Erreur pour le modèle {model_name} : {e}")

▶️ Testing model: tblard/tf-allocine


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
Device set to use 0



📊 Résultats : tblard/tf-allocine
              precision    recall  f1-score   support

     negatif       0.90      0.72      0.80        61
     positif       0.67      0.87      0.76        39

    accuracy                           0.78       100
   macro avg       0.78      0.80      0.78       100
weighted avg       0.81      0.78      0.78       100

▶️ Testing model: nlptown/bert-base-multilingual-uncased-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : nlptown/bert-base-multilingual-uncased-sentiment
              precision    recall  f1-score   support

     negatif       0.88      0.93      0.90        61
     positif       0.89      0.79      0.84        39

    accuracy                           0.88       100
   macro avg       0.88      0.86      0.87       100
weighted avg       0.88      0.88      0.88       100

▶️ Testing model: cardiffnlp/twitter-xlm-roberta-base-sentiment


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



📊 Résultats : cardiffnlp/twitter-xlm-roberta-base-sentiment
              precision    recall  f1-score   support

     negatif       0.61      1.00      0.76        61
     positif       0.00      0.00      0.00        39

    accuracy                           0.61       100
   macro avg       0.30      0.50      0.38       100
weighted avg       0.37      0.61      0.46       100



In [ ]:
# Liste des modèles disponibles
model_cols = {
    "tblard/tf-allocine": "pred_tblard/tf-allocine",
    "nlptown/bert-base-multilingual-uncased-sentiment": "pred_nlptown/bert-base-multilingual-uncased-sentiment",
}

for model_name, col_pred in model_cols.items():
    print(f"\n=== Analyse des erreurs pour {model_name} sur texte brut (comment) ===")

    df_fp_fn = sample_df[["comment", "label", col_pred]].copy()
    df_fp_fn["true_label_str"] = df_fp_fn["label"].apply(lambda x: "positif" if x == 1 else "negatif")

    # Faux positifs = prédit positif mais vrai négatif
    fp = df_fp_fn[(df_fp_fn["true_label_str"] == "negatif") & (df_fp_fn[col_pred] == "positif")]
    fn = df_fp_fn[(df_fp_fn["true_label_str"] == "positif") & (df_fp_fn[col_pred] == "negatif")]

    print(f"\n❌ Faux positifs ({len(fp)} exemples):")
    display(fp[["comment", "true_label_str", col_pred]])

    print(f"\n❌ Faux négatifs ({len(fn)} exemples):")
    display(fn[["comment", "true_label_str", col_pred]])



=== Analyse des erreurs pour tblard/tf-allocine sur texte brut (comment) ===

❌ Faux positifs (17 exemples):


,comment,true_label_str,pred_tblard/tf-allocine
570,"J ai commandé le 22 juin,via CDiscount,un appa...",negatif,positif
920,Produit livré à la mauvaise adresse. (à 600 km...,negatif,positif
888,"Ah, la commande 391866377 - 2505261812HB215, u...",negatif,positif
676,Commandé une multiprise murale pour pouvoir br...,negatif,positif
625,Très déçue de ma commande car coffre de rangem...,negatif,positif
813,Premier avis de ma vie sur une plate-forme en ...,negatif,positif
770,"2 COMMANDES, 2 problèmes : 1/ 1ère commande : ...",negatif,positif
10,"Je n'ai jamais été déçue par cdiscount, livrai...",negatif,positif
516,recherche trés pénible : les filtres qui s'ann...,negatif,positif
812,"Pour acheter vous êtes débiter rapidement, si...",negatif,positif



❌ Faux négatifs (5 exemples):


,comment,true_label_str,pred_tblard/tf-allocine
136,Rien à redire,positif,negatif
59,entreprise française même si tous les articles...,positif,negatif
30,J'ai trouvé rapidement ce que je voulais,positif,negatif
406,Aucun probléme rencontré.Fluide.,positif,negatif
725,Nous avons acheté un drone comme jouet pour no...,positif,negatif



=== Analyse des erreurs pour nlptown/bert-base-multilingual-uncased-sentiment sur texte brut (comment) ===

❌ Faux positifs (4 exemples):


,comment,true_label_str,pred_nlptown/bert-base-multilingual-uncased-sentiment
10,"Je n'ai jamais été déçue par cdiscount, livrai...",negatif,positif
516,recherche trés pénible : les filtres qui s'ann...,negatif,positif
823,Au lieu de protéger leur clients et consommate...,negatif,positif
957,"01.05.2025, nouveau client : -10 euros (super ...",negatif,positif



❌ Faux négatifs (8 exemples):


,comment,true_label_str,pred_nlptown/bert-base-multilingual-uncased-sentiment
859,"Jeudi dernier, j'ai acheté un gros électro mén...",positif,negatif
372,simplifie le choix des articles nécessaires av...,positif,negatif
59,entreprise française même si tous les articles...,positif,negatif
30,J'ai trouvé rapidement ce que je voulais,positif,negatif
406,Aucun probléme rencontré.Fluide.,positif,negatif
725,Nous avons acheté un drone comme jouet pour no...,positif,negatif
313,très satisfaite de cette cigarette électroniqu...,positif,negatif
213,"Livraison rapide, beaucoup de choix de produit...",positif,negatif


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
print(sample_df.columns.tolist())


['comment', 'rating', 'label', 'clean_comment', 'lemmatized', 'pred_tblard/tf-allocine', 'pred_nlptown/bert-base-multilingual-uncased-sentiment', 'pred_cardiffnlp/twitter-xlm-roberta-base-sentiment', 'pred_tblard/tf-allocine_clean_comment', 'pred_nlptown/bert-base-multilingual-uncased-sentiment_clean_comment', 'pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_clean_comment', 'pred_tblard/tf-allocine_lemmatized', 'pred_nlptown/bert-base-multilingual-uncased-sentiment_lemmatized', 'pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_lemmatized']


## 🧪 Comparaison des prédictions du modèle tblard/tf-allocine entre les 3 versions de texte

In [ ]:
# 🧪 Comparaison des prédictions du modèle tblard/tf-allocine entre les 3 versions de texte
models = [ "pred_tblard/tf-allocine","pred_nlptown/bert-base-multilingual-uncased-sentiment"]

for model_base in models:
    cols = {
        "comment": model_base,
        "clean_comment": f"{model_base}_clean_comment",
        "lemmatized": f"{model_base}_lemmatized"
    }

    comparison = sample_df[["comment", "clean_comment", "lemmatized", "label"] + list(cols.values())].copy()
    comparison["true_label_str"] = comparison["label"].apply(lambda x: "positif" if x == 1 else "negatif")

    # ✅ Colonne qui détecte si les prédictions changent
    comparison["prediction_change"] = comparison[list(cols.values())].nunique(axis=1) > 1

    # 🔍 Filtrer les cas où la prédiction change
    changed = comparison[comparison["prediction_change"]]

    print(f"\n🔁 {len(changed)} commentaires ont eu des prédictions différentes selon le prétraitement.")
    display(changed[["comment", "true_label_str"] + list(cols.values())])

    print("------------------------------------------------------------------")



🔁 17 commentaires ont eu des prédictions différentes selon le prétraitement.


,comment,true_label_str,pred_tblard/tf-allocine,pred_tblard/tf-allocine_clean_comment,pred_tblard/tf-allocine_lemmatized
570,"J ai commandé le 22 juin,via CDiscount,un appa...",negatif,positif,negatif,positif
920,Produit livré à la mauvaise adresse. (à 600 km...,negatif,positif,negatif,positif
676,Commandé une multiprise murale pour pouvoir br...,negatif,positif,negatif,positif
625,Très déçue de ma commande car coffre de rangem...,negatif,positif,negatif,positif
813,Premier avis de ma vie sur une plate-forme en ...,negatif,positif,negatif,positif
770,"2 COMMANDES, 2 problèmes : 1/ 1ère commande : ...",negatif,positif,negatif,positif
59,entreprise française même si tous les articles...,positif,negatif,positif,negatif
30,J'ai trouvé rapidement ce que je voulais,positif,negatif,positif,negatif
516,recherche trés pénible : les filtres qui s'ann...,negatif,positif,negatif,positif
812,"Pour acheter vous êtes débiter rapidement, si...",negatif,positif,negatif,positif


------------------------------------------------------------------

🔁 7 commentaires ont eu des prédictions différentes selon le prétraitement.


,comment,true_label_str,pred_nlptown/bert-base-multilingual-uncased-sentiment,pred_nlptown/bert-base-multilingual-uncased-sentiment_clean_comment,pred_nlptown/bert-base-multilingual-uncased-sentiment_lemmatized
372,simplifie le choix des articles nécessaires av...,positif,negatif,positif,negatif
676,Commandé une multiprise murale pour pouvoir br...,negatif,negatif,positif,negatif
813,Premier avis de ma vie sur une plate-forme en ...,negatif,negatif,positif,negatif
671,Ayant eu des soucis avec un vendeur concernant...,positif,positif,negatif,positif
30,J'ai trouvé rapidement ce que je voulais,positif,negatif,positif,negatif
516,recherche trés pénible : les filtres qui s'ann...,negatif,positif,negatif,positif
313,très satisfaite de cette cigarette électroniqu...,positif,negatif,positif,negatif


------------------------------------------------------------------


## Faux positifs corrigés après nettoyage :

In [ ]:
# 📌 Nom du modèle à analyser
models = [ "pred_tblard/tf-allocine","pred_nlptown/bert-base-multilingual-uncased-sentiment"]


# 🔁 Noms des colonnes selon version du texte
cols = {
    "brut": model_base,
    "clean": f"{model_base}_clean_comment",
    "lemm": f"{model_base}_lemmatized"
}
for model_base in models:
    # Sous-échantillon avec prédictions et texte
    df_fp = sample_df[["comment", "clean_comment", "lemmatized", "label"] + list(cols.values())].copy()

    # Calculer les FP dans chaque version
    for v, col in cols.items():
        df_fp[f"FP_{v}"] = (df_fp["label"] == 0) & (df_fp[col] == "positif")

    # Trouver les cas où brut = FP mais les autres ≠ FP
    df_fp["FP_corrigé_clean"] = df_fp["FP_brut"] & ~df_fp["FP_clean"]
    df_fp["FP_corrigé_lemm"] = df_fp["FP_brut"] & ~df_fp["FP_lemm"]

    # Résumés
    print(f"🔎 Faux positifs dans le texte brut : {df_fp['FP_brut'].sum()}")
    print(f"✅ Corrigés après nettoyage      : {df_fp['FP_corrigé_clean'].sum()}")
    print(f"✅ Corrigés après lemmatisation : {df_fp['FP_corrigé_lemm'].sum()}")

    # Affichage des exemples corrigés
    print("\n✅ Faux positifs corrigés après nettoyage :")
    display(df_fp[df_fp["FP_corrigé_clean"]][["comment", "label", cols["brut"], cols["clean"]]])

    print("\n✅ Faux positifs corrigés après lemmatisation :")
    display(df_fp[df_fp["FP_corrigé_lemm"]][["comment", "label", cols["brut"], cols["lemm"]]])


🔎 Faux positifs dans le texte brut : 17
✅ Corrigés après nettoyage      : 14
✅ Corrigés après lemmatisation : 0

✅ Faux positifs corrigés après nettoyage :


,comment,label,pred_tblard/tf-allocine,pred_tblard/tf-allocine_clean_comment
570,"J ai commandé le 22 juin,via CDiscount,un appa...",0,positif,negatif
920,Produit livré à la mauvaise adresse. (à 600 km...,0,positif,negatif
676,Commandé une multiprise murale pour pouvoir br...,0,positif,negatif
625,Très déçue de ma commande car coffre de rangem...,0,positif,negatif
813,Premier avis de ma vie sur une plate-forme en ...,0,positif,negatif
770,"2 COMMANDES, 2 problèmes : 1/ 1ère commande : ...",0,positif,negatif
516,recherche trés pénible : les filtres qui s'ann...,0,positif,negatif
812,"Pour acheter vous êtes débiter rapidement, si...",0,positif,negatif
554,Cafetière commandé le 25 mai nous sommes les 1...,0,positif,negatif
834,"Comme beaucoup apparement, précommande de la S...",0,positif,negatif



✅ Faux positifs corrigés après lemmatisation :


,comment,label,pred_tblard/tf-allocine,pred_tblard/tf-allocine_lemmatized


In [ ]:
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# Récupérer les prédictions
for version in [ "clean_comment", "lemmatized"]:
    col = f"pred_{model_name}_{version}"
    print(f"\n📉 Analyse des sorties du modèle Cardiff sur {version}")
    display(sample_df[[version, "label", col]].value_counts(col).reset_index(name="count"))

    # Filtrer les cas positifs mal classés
    false_neg = sample_df[(sample_df["label"] == 1) & (sample_df[col] == "negatif")]
    print(f"\n❌ Faux négatifs détectés ({len(false_neg)} cas) :")
    display(false_neg[[version, col]])


📉 Analyse des sorties du modèle Cardiff sur clean_comment


,pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_clean_comment,count
0,negatif,100



❌ Faux négatifs détectés (39 cas) :


,clean_comment,pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_clean_comment
431,simple clair,negatif
859,jeudi dernier jai achet un gros lectro mnager ...,negatif
451,rapide efficace,negatif
76,rapide et efficace,negatif
372,simplifie le choix des articles ncessaires ave...,negatif
136,rien redire,negatif
158,les bons dachat avec la carte cdiscount sont t...,negatif
291,la facilit naviguer,negatif
356,facile clair et net,negatif
107,jai trouv rapidement le produit recherch avec ...,negatif



📉 Analyse des sorties du modèle Cardiff sur lemmatized


,pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_lemmatized,count
0,negatif,100



❌ Faux négatifs détectés (39 cas) :


,lemmatized,pred_cardiffnlp/twitter-xlm-roberta-base-sentiment_lemmatized
431,simple clair,negatif
859,jeudi dernier jai achet gros lectro mnager cha...,negatif
451,rapide efficace,negatif
76,rapide efficace,negatif
372,simplifier choix article ncessaire avis panel ...,negatif
136,rien redire,negatif
158,bon dachat carte cdiscount trs intressant,negatif
291,facilit naviguer,negatif
356,facile clair net,negatif
107,jai trouv rapidement produit recherch bon rapp...,negatif
